In [2]:
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Sequential, BatchNorm1d #, ReLU
from torch.nn.functional import relu
from torch.utils.data.sampler import SubsetRandomSampler
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn import ARMAConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import global_add_pool

import argparse
import time
import math
import copy

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdmolops
from rdkit.Chem import Crippen
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdchem import GetPeriodicTable
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.ML.Descriptors import MoleculeDescriptors
import rdkit

from mordred import Calculator, descriptors
from padelpy import padeldescriptor, from_smiles

import xgboost as xgb

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.ensemble import RandomForestClassifier as RF
from sklearn import metrics
from sklearn.svm import SVR



In [3]:
xG_baruta_total_padel_summation = pd.read_csv('../step2-applying_borutapy/xG_baruta_total_padel_summation.csv')

In [4]:
total_smile = pd.read_csv('../dataset/total_smile.csv')
total_smile

,smile_A,mol_fraction_A,smile_B,mol_fraction_B
0,CCCCCCCCCCCCOS(=O)(=O)[O-],0.0,CCCCCCCCCCCCCCCC[N+](C)(C)C,1.0
1,CCCCCCCCCCCCOS(=O)(=O)[O-],0.1,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.9
2,CCCCCCCCCCCCOS(=O)(=O)[O-],0.2,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.8
3,CCCCCCCCCCCCOS(=O)(=O)[O-],0.3,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.7
4,CCCCCCCCCCCCOS(=O)(=O)[O-],0.4,CCCCCCCCCCCCCCCC[N+](C)(C)C,0.6
...,...,...,...,...
974,CCCCCCCCCCCCCCOCC(O)Cn1cc[n+](CCC[n+]2ccn(CC(O...,0.5,CCCCCCCCCCCCCCOCC(O)Cn1cc[n+](CCC[n+]2ccn(CC(O...,0.5
975,CCCCCCCCCCCCCCC(CSCCSCC(CCCCCCCCCCCCCC)[n+]1cc...,0.5,CCCCCCCCCCCCCCC(CSCCSCC(CCCCCCCCCCCCCC)[n+]1cc...,0.5
976,CCCCCCCCCCCCCCC(CSCCCSCC(CCCCCCCCCCCCCC)[n+]1c...,0.5,CCCCCCCCCCCCCCC(CSCCCSCC(CCCCCCCCCCCCCC)[n+]1c...,0.5
977,CCCCCCCCCCCCCCC(CSCCCCSCC(CCCCCCCCCCCCCC)[n+]1...,0.5,CCCCCCCCCCCCCCC(CSCCCCSCC(CCCCCCCCCCCCCC)[n+]1...,0.5


In [6]:
Z_total = pd.read_csv('../step1-generating and merging descriptors/total_property.csv')

In [7]:
# Assuming your dataframes are already loaded
# total_smile: columns = ['smile_A', 'mol_fraction_A', 'smile_B', 'mol_fraction_B']
# Z_total: another dataframe with the same number of rows as total_smile

# Step 1: Identify rows where smile_A == smile_B and mol_fraction_A == mol_fraction_B == 0.5
mask_equal_smiles = (
    (total_smile['smile_A'] == total_smile['smile_B']) &
    (total_smile['mol_fraction_A'] == 0.5) &
    (total_smile['mol_fraction_B'] == 0.5)
)

# Get the indices for these rows
idx_fixed_train = total_smile[mask_equal_smiles].index

# Step 2: Identify the remaining indices
idx_remaining = total_smile[~mask_equal_smiles].index

# Step 3: Randomly split the remaining indices into train and test (80/20)
idx_train_remaining, idx_test = train_test_split(idx_remaining, test_size=0.25, random_state=42)

# Combine fixed train indices with train split
idx_train = idx_fixed_train.union(idx_train_remaining)

# Step 4: Use indices to split both total_smile and Z_total
train_smile = total_smile.loc[idx_train].reset_index(drop=True)
test_smile = total_smile.loc[idx_test].reset_index(drop=True)

Z_train = Z_total.loc[idx_train].reset_index(drop=True)
Z_test = Z_total.loc[idx_test].reset_index(drop=True)

# Optional: check shapes
print(f"Train size: {len(train_smile)}, Test size: {len(test_smile)}")


Train size: 852, Test size: 127


In [24]:
train_smile.to_csv('train_smile.csv', index=False)
test_smile.to_csv('test_smile.csv', index=False)

In [30]:
xG_baruta_test_padel_summation = xG_baruta_total_padel_summation.loc[idx_test]
xG_baruta_train_padel_summation = xG_baruta_total_padel_summation.drop(idx_test)

xG_baruta_test_padel_summation.to_csv('xG_baruta_test_padel_summation.csv', index=False)
xG_baruta_train_padel_summation.to_csv('xG_baruta_train_padel_summation.csv', index=False)

In [31]:
Z_total = pd.read_csv('../step1-generating and merging descriptors/total_property.csv')

# Use these indices to filter the 'Z_total' dataframe
Z_test = Z_total.loc[idx_test]

# Save the rest of the rows in 'Z_train'
Z_train = Z_total.drop(idx_test)

Z_test.to_csv('test_property.csv', index=False)
Z_train.to_csv('train_property.csv', index=False)

Z_train.shape,Z_test.shape

((852, 1), (127, 1))